## Semi-manual updates

In [4]:
import wled_common_client, importlib, datetime
from omegaconf import DictConfig, OmegaConf, ListConfig
importlib.reload(wled_common_client)
from wled_common_client import Wled, Wleds
from scripts.local_env import default_wled_ip
from time import sleep
import wled_listener as wl
wl.wleds.print()
default_wled = wl.wleds.get_by_ip(default_wled_ip())
from IPython.display import display_html, clear_output

def display_dicts(wleds, dicts, fields=["ver"]):
    clear_output(wait=True)
    r = "<br><table><tr><th>WLED</th>"
    for field in fields:
        r += f"<th>{field}</th>"
    r+="</tr>"
    for wled, state in zip(wleds, dicts):
        r += f"<tr><td>{wled}</td>"
        for field in fields:
            if field in state.keys():
                value = state[field]
                r += f"<td>{value}</td>"
            else:
                r += f"<td>NO {field}</td>"
        r += "</tr>"
    r += "</table>"
    display_html(r, raw=True)


WLED 'WLED-frame-Cubes' at 192.168.0.12
WLED 'WLED-frame-Stroop' at 192.168.0.23
WLED 'WLED-frame-Three-Colors' at 192.168.0.6


In [5]:
display_dicts(wl.wleds, wl.wleds.get_json_info(), fields=["ver", "u"])

WLED,ver,u
WLED 'WLED-frame-Cubes' at 192.168.0.12,0.13.0-b6,"{'NTP': {'source': 20}, 'random': {'seed': 1337}}"
WLED 'WLED-frame-Stroop' at 192.168.0.23,0.13.0-b5,NO u
WLED 'WLED-frame-Three-Colors' at 192.168.0.6,0.13.0-b5,NO u


In [26]:
default_wled.set_preset_udp(17)

In [6]:
wl.wleds

[WLED 'WLED-frame-Cubes' at 192.168.0.12, WLED 'WLED-frame-Stroop' at 192.168.0.23, WLED 'WLED-frame-Three-Colors' at 192.168.0.6]

In [27]:
from IPython.display import display_html, clear_output
for i in range(1000):
    clear_output(wait=True)
    state = wl.wleds["WLED-Dima-Office"].get_json_state()
    r = f"{datetime.datetime.now()}<br>"
    r += f"last sent: {state['last_sent_string']}<br>"
    r += f"last next: {state['last_next_string']}<br>"
    for wled, state in zip(wl.wleds, wl.wleds.get_json_state()):
        if "random_seed" in state.keys():
            r += f"<br>Seed {state['random_seed']} for {wled}"
        else:
            r += f"<br>NO SEED KNOWN for {wled}"
    display_html(r, raw=True)
    sleep(1)

KeyboardInterrupt: 

In [3]:
display_dicts(wl.wleds, wl.wleds.get_json_info(), fields=["ver", "u"])

WLED,ver,u
WLED 'WLED-frame-Cubes' at 192.168.0.12,0.13.0-b6,"{'NTP': {'source': 20}, 'random': {'seed': 1337}}"
WLED 'WLED-frame-Stroop' at 192.168.0.23,0.13.0-b5,NO u
WLED 'WLED-frame-Three-Colors' at 192.168.0.6,0.13.0-b5,NO u


In [3]:
wl.wleds.dump_omegaconf()

[None, None, None, None, None, None, None, None, None]

In [2]:
wl.wleds.reconfig_from_omegaconf(keep_presets=False)

[WLED 'WLED-frame-Cubes' at 192.168.0.12, WLED 'WLED-frame-Stroop' at 192.168.0.23, WLED 'WLED-frame-Three-Colors' at 192.168.0.6]

In [13]:
Wled.reconfig_from_omegaconf(default_wled, keep_presets=False)

In [17]:
default_wled.cfg["timers"]

{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0},
 'ins': [{'en': 0, 'hour': 9, 'min': 55, 'macro': 23, 'dow': 127},
  {'en': 0, 'hour': 23, 'min': 0, 'macro': 12, 'dow': 127},
  {'en': 0, 'hour': 19, 'min': 13, 'macro': 12, 'dow': 127},
  {'en': 0, 'hour': 19, 'min': 15, 'macro': 23, 'dow': 127},
  {'en': 0, 'hour': 255, 'min': 0, 'macro': 0, 'dow': 127},
  {'en': 0, 'hour': 255, 'min': 0, 'macro': 0, 'dow': 127}]}

In [ ]:
for wled in wleds:
    print (wled)
    wled.set_effect(0)
    wled.set_random_seed(42)
    wled.update_time()
    print("OK")
wleds["WLED-Dima-Office"].set_fake_NTP(130)
wleds["WLED-Dima-Office"].set_preset(23)

WLED 'WLED kitchen' at 192.168.0.121
OK
WLED 'WLED-Dima-Office' at 192.168.0.122
OK
WLED 'WLED-UV' at 192.168.0.10
OK
WLED 'WLED-frame-clock' at 192.168.0.15
OK
WLED 'WLED-table-lamp' at 192.168.0.113
OK
WLED 'WLED-tube-1' at 192.168.0.28
OK


In [ ]:
for wled in wleds:
    print (wled)
    wled.set_effect(0)
    wled.set_random_seed(42)
    wled.update_time()
    print("OK")
wleds["WLED-Dima-Office"].set_fake_NTP(130)
wleds["WLED-Dima-Office"].set_preset(23)

WLED 'WLED kitchen' at 192.168.0.121
OK
WLED 'WLED-Dima-Office' at 192.168.0.122
OK
WLED 'WLED-UV' at 192.168.0.10
OK
WLED 'WLED-frame-clock' at 192.168.0.15
OK
WLED 'WLED-table-lamp' at 192.168.0.113
OK
WLED 'WLED-tube-1' at 192.168.0.28
OK


In [145]:
# wleds["WLED-Dima-Office"].update_firmware("../../Arduino/WLED/build_output/firmware/d1_mini.bin")

<Response [200]>

In [163]:
wleds.wleds.remove(wleds["WLED-table-lamp"])

In [5]:
print("FIRMWARE update of all the Wleds")
# wleds.update_firmware("../../Arduino/WLED/build_output/firmware/d1_mini.bin")
# default_wled.update_firmware("../../Arduino/WLED/build_output/firmware/d1_mini.bin")


FIRMWARE update of all the Wleds


<Response [200]>

In [19]:
Wled.reconfig_from_omegaconf(wleds["WLED-Xmas-tree"])

In [5]:
for wled in wleds:
    print(wled.name)
    wled.dump_omegaconf()

WLED kitchen
WLED-D-mini-3
WLED-Dima-Office


UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 2554: character maps to <undefined>

In [176]:
for wled in wleds:
    wled.presets = OmegaConf.to_container(OmegaConf.load("default_presets.yaml"))
    wled.upload_presets()
    wled.update_time()
wleds.reset()


[200, 200, 200, 200, 200]

In [64]:
for wled in wleds:
    oc_dir = wled.get_fs_dump_dir("omegaconf_source/{name}")
    cfg_yaml = f"{oc_dir}/cfg.yaml"
    ww = Wled.from_omegaconf(additional_confs=[cfg_yaml])
    ww.ip = wled.ip
    print(ww)
    print(ww.cfg["timers"])
    # ww.upload_cfg()

WLED 'WLED kitchen' at 192.168.0.121
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}
WLED 'WLED-D-mini' at 192.168.0.32
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': [{'dow': 127, 'en': 0, 'hour': 10, 'macro': 6, 'min': 0}, {'dow': 127, 'en': 0, 'hour': 23, 'macro': 12, 'min': 40}]}
WLED 'WLED-D-mini-2' at 192.168.0.34
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': [{'dow': 127, 'en': 0, 'hour': 10, 'macro': 6, 'min': 0}, {'dow': 127, 'en': 0, 'hour': 23, 'macro': 12, 'min': 40}]}
WLED 'WLED-Dima-Office' at 192.168.0.122
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': [{'dow': 127, 'en': 0, 'hour': 10, 'macro': 6, 'min': 0}, {'dow': 127, 'en': 0, 'hour': 23, 'macro': 12, 'min': 40}]}
WLED 'WLED-frame-clock' at 192.168.0.15
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': [{'dow': 127, 'en': 0, 'hour': 10, 'macro': 6, 'min': 0}, {'dow': 127, 'en': 0, 'hour': 23, 'macro': 12, 'min': 40}]}


In [35]:
for wled in wleds:
    print(wled)
    print(wled.cfg["timers"])

WLED 'WLED kitchen' at 192.168.0.121
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}
WLED 'WLED-D-mini' at 192.168.0.32
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}
WLED 'WLED-D-mini-2' at 192.168.0.34
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}
WLED 'WLED-Dima-Office' at 192.168.0.122
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}
WLED 'WLED-frame-clock' at 192.168.0.15
{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}


In [22]:
d_mini = wleds["WLED-Dima-Office"]

In [23]:
d_mini.cache_fs()

In [24]:
d_mini.cfg

{'rev': [1, 0],
 'vid': 2107100,
 'id': {'mdns': 'wled-dima-office',
  'name': 'WLED-Dima-Office',
  'inv': 'Light'},
 'nw': {'ins': [{'ssid': 'Vadkovsky_TV',
    'pskl': 15,
    'ip': [192, 168, 0, 121],
    'gw': [0, 0, 0, 0],
    'sn': [255, 255, 255, 0]}]},
 'ap': {'ssid': 'WLED-AP',
  'pskl': 8,
  'chan': 1,
  'hide': 0,
  'behav': 0,
  'ip': [4, 3, 2, 1]},
 'wifi': {'sleep': False},
 'hw': {'led': {'total': 200,
   'maxpwr': 6000,
   'ledma': 55,
   'rgbwm': 3,
   'ins': [{'start': 0,
     'len': 200,
     'pin': [4],
     'order': 0,
     'rev': False,
     'skip': 0,
     'type': 22}]},
  'btn': {'max': 2,
   'ins': [{'type': 2, 'pin': [0], 'macros': [0, 0, 0]},
    {'type': 0, 'pin': [-1], 'macros': [0, 0, 0]}],
   'tt': 32,
   'mqtt': False},
  'ir': {'pin': -1, 'type': 0},
  'relay': {'pin': 12, 'rev': False}},
 'light': {'scale-bri': 100,
  'pal-mode': 0,
  'gc': {'bri': 1, 'col': 2.8},
  'tr': {'mode': True, 'dur': 7, 'pal': 0},
  'nl': {'mode': 1, 'dur': 60, 'tbri': 0, 'm

In [16]:
for wled in wleds:
    wled.reset()

In [9]:
for wled in wleds:
    wled.update_time()

In [15]:
ww.cfg["timers"]

{'cntdwn': {'goal': [20, 1, 1, 0, 0, 0], 'macro': 0}, 'ins': []}

In [ ]:
for wled in wleds:
    oc_dir = wled.get_fs_dump_dir(wled_common_client.OMEGACONF_DUMP_DIR)
    cfg_yaml = f"{oc_dir}/cfg.yaml"
    presets_yaml = f"{oc_dir}/presets.yaml"
    wled.presets = OmegaConf.load(presets_yaml)
    wled.cfg = OmegaConf.load(presets_yaml)

In [ ]:
yaml.load()

In [28]:
math.ceil(1000/ (512//3))

6

## DMX interface

In [20]:
import wled_common_client, importlib, datetime, time
from omegaconf import DictConfig, OmegaConf, ListConfig
importlib.reload(wled_common_client)
from wled_common_client import Wled, Wleds
from scripts.local_env import default_wled_ip
from time import sleep
wleds = Wleds.from_one_ip(default_wled_ip(), )
wleds.print()
default_wled = wleds.get_by_ip(default_wled_ip())

WLED 'WLED kitchen' at 192.168.0.121
WLED 'WLED-Dima-Office' at 192.168.0.122
WLED 'WLED-UV' at 192.168.0.14
WLED 'WLED-Xmas-tree' at 192.168.0.7
WLED 'WLED-frame-L1' at 192.168.0.19
WLED 'WLED-frame-L2' at 192.168.0.11
WLED 'WLED-frame-L3' at 192.168.0.13
WLED 'WLED-frame-clock' at 192.168.0.15
WLED 'WLED-tube-1' at 192.168.0.12


In [4]:
f"{datetime.datetime.now()}"

'2022-02-17 22:25:02.898940'

In [13]:
wl1.reset()

200

In [21]:
# wl1 = wleds["WLED-Dima-Office"]
wl1 = wleds["WLED-tube-1"]

In [32]:
wl1.dmx.start()

In [33]:
wl1.dmx.sender._sender_handler.socket.fps

30

In [23]:
data = [255, 255,0] * (wl1.dmx.n_leds//2 )
data += [0, 255,0] * (len(data) - wl1.dmx.n_leds)

In [24]:
for i in range(1000):
    wl1.dmx.set_data(data)
    time.sleep(0.01)
    for i in range(3):
        data.insert(0, data.pop())

In [10]:
wl1.dmx.set_data(data)

In [31]:
wl1.dmx.stop()

## DMX experiments

In [30]:
import sacn
import time
import math

In [36]:
# sender = sacn.sACNsender(bind_address="192.168.0.16")  # provide an IP-Address to bind to if you are using Windows and want to use multicast
sender = sacn.sACNsender()  # provide an IP-Address to bind to if you are using Windows and want to use multicast
sender.start()  # start the sending thread

In [42]:

sender.activate_output(1)  # start sending out data in the 1st universe
# sender.activate_output(2)  # start sending out data in the 1st universe
sender[1].multicast = True  # set multicast to True
sender[1].fps=10
# sender[2].multicast = True  # set multicast to True
# sender.destination = "192.168.0.14"  # or provide unicast information.
# sender[1].destination = "192.168.0.122"  # or provide unicast information.
# sender[2].destination = sender[1].destination  # or provide unicast information.
# sender[1].fps = 60
# Keep in mind that if multicast is on, unicast is not used



In [41]:
sender[1].dmx_data = tuple([0, 0, 0] * 100 + [255, 0, 0] * 70 )  # some test DMX data


In [40]:
sender[2].dmx_data = tuple([0, 0, 0]*10 + [255, 0, 0]*166)  # some test DMX data


AttributeError: 'NoneType' object has no attribute 'dmx_data'

In [35]:
sender.stop()  # do not forget to stop the sender

## Tidying up presets

In [158]:
import json
from ruamel.yaml import YAML
yaml = YAML()

In [119]:
with open("config_dump/WLED-Dima-Office/presets.json") as js:
    p = yaml.load(js)

In [120]:
# yaml.compact_seq_seq = True
# yaml.compact_seq_map = False
yaml.width = 10**7
yaml.top_level_colon_align = True


In [121]:
p.fa.set_block_style()
for k in p.keys():
    p[k].fa.set_block_style()

In [122]:
with open("default_presets.yaml", "w") as y:
    yaml.dump(p, y)


# MIDI

In [1]:
import sys
import os

import pygame as pg
import pygame.midi


def print_device_info():
    pygame.midi.init()
    _print_device_info()
    pygame.midi.quit()


def _print_device_info():
    for i in range(pygame.midi.get_count()):
        r = pygame.midi.get_device_info(i)
        (interf, name, input, output, opened) = r

        in_out = ""
        if input:
            in_out = "(input)"
        if output:
            in_out = "(output)"

        print(
            "%2i: interface :%s:, name :%s:, opened :%s:  %s"
            % (i, interf, name, opened, in_out)
        )


def input_main(device_id=None):
    pg.init()
    pg.fastevent.init()
    event_get = pg.fastevent.get
    event_post = pg.fastevent.post

    pygame.midi.init()

    _print_device_info()

    if device_id is None:
        input_id = pygame.midi.get_default_input_id()
    else:
        input_id = device_id

    print("using input_id :%s:" % input_id)
    i = pygame.midi.Input(input_id)

    pg.display.set_mode((1, 1))

    going = True
    while going:
        events = event_get()
        for e in events:
            if e.type in [pg.QUIT]:
                going = False
            if e.type in [pg.KEYDOWN]:
                going = False
            if e.type in [pygame.midi.MIDIIN]:
                print(e)

        if i.poll():
            midi_events = i.read(10)
            # convert them into pygame events.
            midi_evs = pygame.midi.midis2events(midi_events, i.device_id)

            for m_e in midi_evs:
                event_post(m_e)

    del i
    pygame.midi.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
input_main()

## Pure python

In [1]:
from shows import *

In [2]:
ColorCycle.__name__

'ColorCycle'

In [11]:
s = {("a", 3), ("b", 5)}

In [4]:
s.add(("c", 5))

In [5]:
s

{('a', 3), ('b', 5), ('c', 5)}

In [6]:
("b", 5) in s

True

In [7]:
import time

In [9]:
time.time()

1643546191.9331198

In [13]:
import os, sys, platform

In [14]:
python_version = '%s.%s' % (sys.version_info[0], sys.version_info[1])
python_full_version = sys.version.split()[0]
os.name = os.name
sys.platform = sys.platform
platform.version = platform.version()
platform.machine = platform.machine()
platform.python_implementation = platform.python_implementation()

In [15]:
os.name

'posix'

In [12]:
import preset_manager, importlib
importlib.reload(preset_manager)

<module 'preset_manager' from '/home/pi/Svetlitsa_control/scripts/../preset_manager.py'>

In [14]:
preset_manager.get_udp_kwargs(17)

{'col': [0, 255, 0],
 'secondary_color': [0, 255, 200],
 'tertiary_color': [8, 255, 0],
 'fx': 0,
 'fx_speed': 173,
 'fx_intensity': 86,
 'palette': 0,
 'sync_groups': [1]}

In [7]:
tertiary_color = [1]
tertiary_color += ([0] * (4 - len(tertiary_color) ))
tertiary_color

[1, 0, 0, 0]

In [1]:
from enum import Enum
class Some(Enum):
    a = "boo"

In [3]:
Some.a.value

'boo'